# 整体结构

In [42]:
from docx import Document
import pdfplumber
import sys
import os
from pathlib import Path

### word数据读取

In [5]:
def read_docx(docx_path):

    doc = Document(docx_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return ''.join(full_text)

### PDF数据读取和清洗

In [1]:
import re

def clean_pdf_text(text):
    """
    专业法律条文清洗函数
    处理特点：
    1. 保留完整的条款编号（如"第三十四条"）
    2. 规范化法律条文中的特殊格式（如"（一）"等）
    3. 处理中文数字和阿拉伯数字混用情况
    4. 智能分段和换行处理
    """
    # 1. 去除页眉页脚和网页信息
    text = re.sub(r'\d{4}/\d{1,2}/\d{1,2} \d{1,2}:\d{2}', '', text)  # 去除日期时间
    text = re.sub(r'第.*?国务院公报.*?政府网', '', text)
    text = re.sub(r'\d+/\d+', '', text)  # 去除页码
    
    # 2. 条款编号规范化处理
    text = re.sub(r'第([一二三四五六七八九十百千]+)条', r'\n第\1条 ', text)
    
    # 3. 处理中文数字和阿拉伯数字混用（如"第三十四"和"34"）
    text = re.sub(r'第([0-9]+)条', lambda m: f'第{num2chinese(m.group(1))}条', text)
    
    # 4. 法律条文特殊格式处理
    text = re.sub(r'（([一二三四五六七八九十])）', r'（\1）', text)  # 统一括号格式
    text = re.sub(r'([。；])\s*', r'\1\n', text)  # 句号和分号后换行
    
    # 5. 处理列表项格式
    text = re.sub(r'([（(][一二三四五六七八九十]+[）)])', r'\n\1', text)
    
    # 6. 去除多余空行但保留段落分隔
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'[ \t]+', ' ', text)  # 压缩空格
    
    # 7. 章节标题处理
    text = re.sub(r'第[一二三四五六七八九十]+章\s+.+', r'\n\g<0>\n', text)
    
    # 8. 去网页垃圾字符
    text = re.sub(r'http[s]\S+', '', text)
    text = re.sub(r'©|®|™|•', '', text)
    
    return text.strip()

In [160]:
# import re

# def clean_pdf_text(text):
#     """
#     文本清洗
#     """
#     # 去除特殊字符但保留中文标点
#     text = re.sub(r'[^\w\u4e00-\u9fff，。、；：？！「」『』（）《》【】\s]', '', text)
#     # 规范化空白字符
#     text = re.sub(r'\s+', ' ', text)
#     # 处理错误换行（英文单词）
#     text = re.sub(r'(\w)-\s+(\w)', r'\1\2', text)
#     # 处理错误换行（中文）
#     text = re.sub(r'([\u4e00-\u9fff])\s+([\u4e00-\u9fff])', r'\1\2', text)
#     return text.strip()

In [161]:
def read_pdf(pdf_path):
    """
    基础文本提取
    :param pdf_path: PDF文件路径
    :return: 提取的文本字典 {page_num: text}
    """
    text_dict = {}
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:  # 过滤空页
                text = clean_pdf_text(text)
                text_dict[i+1] = text
                
    return text_dict

### 测试两种文档

In [162]:
docx_demo = r'../../knowledge/raw_knowledge/laws/中华人民共和国招标投标法实施条例.docx'
pdf_demo = r'../../knowledge/raw_knowledge/laws/中华人民共和国财政部令（第74号）_政府采购非招标采购方式管理办法_2014年第9号国务院公报_中国政府网.pdf'
# pdf_demo = r'../../knowledge/raw_knowledge/laws/中华人民共和国财政部令（第94号）政府采购质疑和投诉办法_2018年第13号国务院公报_中国政府网.pdf'

In [163]:
!pwd

/Users/ethanliu/Documents/LLM项目/BidMaster/src/pipline


In [164]:
docxs = read_docx(docx_demo)

In [165]:
docxs[:20]

'中华人民共和国招标投标法实施条例(201'

In [166]:
pdfs = read_pdf(pdf_demo)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [167]:
pdfs.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [168]:
pdfs[2]

'中华⼈民共和国财政部令（第74号） 政府采购⾮招标采购⽅式管理办法__2014年第9号国务院公报_中国政府⽹\n\n（三）达到公开招标数额标准、经批准采⽤⾮公开招标⽅式的货物、服务；\n\n（四）按照招标投标法及其实施条例必须进⾏招标的⼯程建设项⽬以外的政府\n采购⼯程。\n第⼆章 ⼀般规定\n第\n第四四条 条 达到公开招标数额标准的货物、服务采购项⽬，拟采⽤⾮招标采购\n⽅式的，采购⼈应当在采购活动开始前，报经主管预算单位同意后，向设区的\n市、⾃治州以上⼈⺠政府财政部⻔申请批准。\n第\n第五五条 条 根据本办法\n第四条 申请采⽤⾮招标采购⽅式采购的，采购⼈应当向\n财政部⻔提交以下材料并对材料的真实性负责：\n（⼀）采购⼈名称、采购项⽬名称、项⽬概况等项⽬基本情况说明；\n（⼆）项⽬预算⾦额、预算批复⽂件或者资⾦来源证明；\n\n（三）拟申请采⽤的采购⽅式和理由。\n第\n第六六条 条 采购⼈、采购代理机构应当按照政府采购法和本办法的规定组织开\n展⾮招标采购活动，并采取必要措施，保证评审在严格保密的情况下进⾏。\n任何单位和个⼈不得⾮法⼲预、影响评审过程和结果。\n第\n第七七条 条 竞争性谈判⼩组或者询价⼩组由采购⼈代表和评审专家共3⼈以上单\n数组成，其中评审专家⼈数不得少于竞争性谈判⼩组或者询价⼩组成员总数的\n。\n采购⼈不得以评审专家身份参加本部⻔或本单位采购项⽬的评审。\n采购代理\n机构⼈员不得参加本机构代理的采购项⽬的评审。\n达到公开招标数额标准的货物或者服务采购项⽬，或者达到招标规模标准的\n政府采购⼯程，竞争性谈判⼩组或者询价⼩组应当由5⼈以上单数组成。\n采⽤竞争性谈判、询价⽅式采购的政府采购项⽬，评审专家应当从政府采购\n评审专家库内相关专业的专家名单中随机抽取。\n技术复杂、专业性强的竞争性谈\n判采购项⽬，通过随机⽅式难以确定合适的评审专家的，经主管预算单位同意，\n可以⾃⾏选定评审专家。\n技术复杂、专业性强的竞争性谈判采购项⽬，评审专家\n中应当包含1名法律专家。\n第第⼋⼋条条 竞争性谈判⼩组或者询价⼩组在采购活动过程中应当履⾏下列职\n责：\n（⼀）确认或者制定谈判⽂件、询价通知书；\n（⼆）从符合相应资格条件的供应商名单中确定不少于3家的供应商参加谈判\n或者询价；\n\n（三）审查供应商的响应⽂件并作出评价；\

## 向量化及其存储

### 向量化处理

In [171]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('moka-ai/m3e-base')

In [192]:
pdf_texts = list(pdfs.values())
pdf_texts[0]

'中华⼈民共和国财政部令（第74号） 政府采购⾮招标采购⽅式管理办法__2014年第9号国务院公报_中国政府⽹\n中中华华⼈⼈⺠⺠共共和和国国财财政政部部令令\n第 7744 号\n《政府采购⾮招标采购⽅式管理办法》已经2013年10⽉28⽇财政部部务会议审\n议通过，现予公布，⾃2014年2⽉1⽇起施⾏。\n部 ⻓ 楼继伟\n2013年12⽉19⽇\n政政府府采采购购⾮⾮招招标标采采购购⽅⽅式式管管理理办办法法\n第⼀章 总 则\n第第⼀⼀条条 为了规范政府采购⾏为，加强对采⽤⾮招标采购⽅式采购活动的监\n督管理，维护国家利益、社会公共利益和政府采购当事⼈的合法权益，依据《中\n华⼈⺠共和国政府采购法》（以下简称政府采购法）和其他法律、⾏政法规的有关\n规定，制定本办法。\n第第⼆⼆条条 采购⼈、采购代理机构采⽤⾮招标采购⽅式采购货物、⼯程和服务\n的，适⽤本办法。\n本办法所称⾮招标采购⽅式，是指竞争性谈判、单⼀来源采购和询价采购⽅\n式。\n竞争性谈判是指谈判⼩组与符合资格条件的供应商就采购货物、⼯程和服务\n事宜进⾏谈判，供应商按照谈判⽂件的要求提交响应⽂件和最后报价，采购⼈从\n谈判⼩组提出的成交候选⼈中确定成交供应商的采购⽅式。\n单⼀来源采购是指采购⼈从某⼀特定供应商处采购货物、⼯程和服务的采购\n⽅式。\n询价是指询价⼩组向符合资格条件的供应商发出采购货物询价通知书，要求\n供应商⼀次报出不得更改的价格，采购⼈从询价⼩组提出的成交候选⼈中确定成\n交供应商的采购⽅式。\n第\n第三三条 条 采购⼈、采购代理机构采购以下货物、⼯程和服务之⼀的，可以采\n⽤竞争性谈判、单⼀来源采购⽅式采购；\n采购货物的，还可以采⽤询价采购⽅\n式：\n（⼀）依法制定的集中采购⽬录以内，且未达到公开招标数额标准的货物、服\n务；\n（⼆）依法制定的集中采购⽬录以外、采购限额标准以上，且未达到公开招标\n数额标准的货物、服务；'

In [194]:
#Sentences are encoded by calling model.encode()
pdf_embeddings = model.encode(pdf_texts)

In [197]:
len(pdf_embeddings)

10

In [198]:
text_len = 512
review_len = 20
docx_texts = []

In [199]:
for idx in range(0, len(docxs), text_len) :
    if idx == 0:
        docx_texts.append(docxs[: idx + text_len])
    else:
        docx_texts.append(docxs[idx - review_len: idx + text_len])

In [200]:
len(docx_texts), docx_texts[-1]

(24,
 '法干涉评标活动，影响中标结果；(三)以其他方式非法干涉招标投标活动。第八十一条\u3000依法必须进行招标的项目的招标投标活动违反招标投标法和本条例的规定，对中标结果造成实质性影响，且不能采取补救措施予以纠正的，招标、投标、中标无效，应当依法重新招标或者评标。第七章\u3000附则第八十二条\u3000招标投标协会按照依法制定的章程开展活动，加强行业自律和服务。第八十三条\u3000政府采购的法律、行政法规对政府采购货物、服务的招标投标另有规定的，从其规定。第八十四条\u3000本条例自2012年2月1日起施行。')

In [201]:
docx_embeddings = model.encode(docx_texts)

In [202]:
len(docx_embeddings)

24

### 向量化存储

In [204]:
from pymilvus import MilvusClient

In [205]:
milvus_client = MilvusClient(uri="../../knowledge/vector_knowledge/laws.db")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [208]:
embedding_dim = len(pdf_embeddings[0])
collection_name = "laws_collection"

In [209]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Supported values are (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`). See https://milvus.io/docs/consistency.md#Consistency-Level for more details.
)

In [210]:
data = []
idx = 0
for sentence, embedding in zip(pdf_texts, pdf_embeddings):
    data.append({"id": idx, "sentence": sentence, "vector": embedding, "from_doc": None})
    idx += 1


for sentence, embedding in zip(docx_texts, docx_embeddings):
    data.append({"id": idx, "sentence": sentence, "vector": embedding, "from_doc": None})
    idx += 1   

In [211]:
milvus_client.insert(collection_name=collection_name, data=data)

{'insert_count': 34, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], 'cost': 0}

## 匹配测试

In [228]:
question = '投标人用他人的名义进行投标或者以其他方式弄虚作假骗取中标的情况怎么处理？'

In [236]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        model.encode(question)
    ],
    
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["sentence"],  # Return the sentence field
)

In [237]:
search_res[0]

[{'id': 29, 'distance': 328.5481872558594, 'entity': {'sentence': '标；(二)3年内2次以上使用他人名义投标；(三)弄虚作假骗取中标给招标人造成直接经济损失30万元以上；(四)其他弄虚作假骗取中标情节严重的行为。投标人自本条第二款规定的处罚执行期限届满之日起3年内又有该款所列违法行为之一的，或者弄虚作假骗取中标情节特别严重的，由工商行政管理机关吊销营业执照。第六十九条\u3000出让或者出租资格、资质证书供他人投标的，依照法律、行政法规的规定给予行政处罚；构成犯罪的，依法追究刑事责任。第七十条\u3000依法必须进行招标的项目的招标人不按照规定组建评标委员会，或者确定、更换评标委员会成员违反招标投标法和本条例规定的，由有关行政监督部门责令改正，可以处10万元以下的罚款，对单位直接负责的主管人员和其他直接责任人员依法给予处分；违法确定或者更换的评标委员会成员作出的评审结论无效，依法重新进行评审。国家工作人员以任何方式非法干涉选取评标委员会成员的，依照本条例第八十条的规定追究法律责任。第七十一条\u3000评标委员会成员有下列行为之一的，由有关行政监督部门责令改正；情节严重的，禁止其在一定期限内参加依法必须进行招标的项目的评标；情节特别严重的，取消其担任评标委员会成员的资格：(一)应当回避而不回避；(二)擅离职守；(三)不按照招标文件规定的评标标准'}}, {'id': 30, 'distance': 322.4468994140625, 'entity': {'sentence': '职守；(三)不按照招标文件规定的评标标准和方法评标；(四)私下接触投标人；(五)向招标人征询确定中标人的意向或者接受任何单位或者个人明示或者暗示提出的倾向或者排斥特定投标人的要求；(六)对依法应当否决的投标不提出否决意见；(七)暗示或者诱导投标人作出澄清、说明或者接受投标人主动提出的澄清、说明；(八)其他不客观、不公正履行职务的行为。第七十二条\u3000评标委员会成员收受投标人的财物或者其他好处的，没收收受的财物，处3000元以上5万元以下的罚款，取消担任评标委员会成员的资格，不得再参加依法必须进行招标的项目的评标；构成犯罪的，依法追究刑事责任。第七十三条\u3000依法必须进行招标的项目的招标人有下列情形之一的，由有关

In [238]:
search_res[0][0]['entity']['sentence']

'标；(二)3年内2次以上使用他人名义投标；(三)弄虚作假骗取中标给招标人造成直接经济损失30万元以上；(四)其他弄虚作假骗取中标情节严重的行为。投标人自本条第二款规定的处罚执行期限届满之日起3年内又有该款所列违法行为之一的，或者弄虚作假骗取中标情节特别严重的，由工商行政管理机关吊销营业执照。第六十九条\u3000出让或者出租资格、资质证书供他人投标的，依照法律、行政法规的规定给予行政处罚；构成犯罪的，依法追究刑事责任。第七十条\u3000依法必须进行招标的项目的招标人不按照规定组建评标委员会，或者确定、更换评标委员会成员违反招标投标法和本条例规定的，由有关行政监督部门责令改正，可以处10万元以下的罚款，对单位直接负责的主管人员和其他直接责任人员依法给予处分；违法确定或者更换的评标委员会成员作出的评审结论无效，依法重新进行评审。国家工作人员以任何方式非法干涉选取评标委员会成员的，依照本条例第八十条的规定追究法律责任。第七十一条\u3000评标委员会成员有下列行为之一的，由有关行政监督部门责令改正；情节严重的，禁止其在一定期限内参加依法必须进行招标的项目的评标；情节特别严重的，取消其担任评标委员会成员的资格：(一)应当回避而不回避；(二)擅离职守；(三)不按照招标文件规定的评标标准'

In [239]:
context = "\n".join(
    [res['entity']['sentence'] for res in search_res[0]]
)

In [240]:
context

'标；(二)3年内2次以上使用他人名义投标；(三)弄虚作假骗取中标给招标人造成直接经济损失30万元以上；(四)其他弄虚作假骗取中标情节严重的行为。投标人自本条第二款规定的处罚执行期限届满之日起3年内又有该款所列违法行为之一的，或者弄虚作假骗取中标情节特别严重的，由工商行政管理机关吊销营业执照。第六十九条\u3000出让或者出租资格、资质证书供他人投标的，依照法律、行政法规的规定给予行政处罚；构成犯罪的，依法追究刑事责任。第七十条\u3000依法必须进行招标的项目的招标人不按照规定组建评标委员会，或者确定、更换评标委员会成员违反招标投标法和本条例规定的，由有关行政监督部门责令改正，可以处10万元以下的罚款，对单位直接负责的主管人员和其他直接责任人员依法给予处分；违法确定或者更换的评标委员会成员作出的评审结论无效，依法重新进行评审。国家工作人员以任何方式非法干涉选取评标委员会成员的，依照本条例第八十条的规定追究法律责任。第七十一条\u3000评标委员会成员有下列行为之一的，由有关行政监督部门责令改正；情节严重的，禁止其在一定期限内参加依法必须进行招标的项目的评标；情节特别严重的，取消其担任评标委员会成员的资格：(一)应当回避而不回避；(二)擅离职守；(三)不按照招标文件规定的评标标准\n职守；(三)不按照招标文件规定的评标标准和方法评标；(四)私下接触投标人；(五)向招标人征询确定中标人的意向或者接受任何单位或者个人明示或者暗示提出的倾向或者排斥特定投标人的要求；(六)对依法应当否决的投标不提出否决意见；(七)暗示或者诱导投标人作出澄清、说明或者接受投标人主动提出的澄清、说明；(八)其他不客观、不公正履行职务的行为。第七十二条\u3000评标委员会成员收受投标人的财物或者其他好处的，没收收受的财物，处3000元以上5万元以下的罚款，取消担任评标委员会成员的资格，不得再参加依法必须进行招标的项目的评标；构成犯罪的，依法追究刑事责任。第七十三条\u3000依法必须进行招标的项目的招标人有下列情形之一的，由有关行政监督部门责令改正，可以处中标项目金额10‰以下的罚款；给他人造成损失的，依法承担赔偿责任；对单位直接负责的主管人员和其他直接责任人员依法给予处分：(一)无正当理由不发出中标通知书；(二)不按照规定确定中标人；(三)中标通知书发出后无正当理由改变中标结果；(四)无正当

In [241]:
SYSTEM_PROMPT = """
Human: 你是一个法律助手，你可以通过所给匹配的法律知识库内容中找到问题的答案并整理
"""
USER_PROMPT = f"""
使用以下包含在<context>标记中的信息片段来回答<question>标记中包含的问题。
<context>
{context}
</context>
<question>
{question}
</question>
"""

In [243]:
from dotenv import load_dotenv
import os

# 加载特定路径的 .env 文件
load_dotenv(".env")

DEEPSEEK_API_KEY = os.getenv("API_KEY")

In [244]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI


client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
    stream=True
)

full_response = ""
for chunk in response:
    if chunk.choices[0].delta.content:  # 检查是否有新内容
        content = chunk.choices[0].delta.content
        print(content, end="", flush=True)  # 逐字打印
        full_response += content

根据提供的法律知识库内容，对投标人使用他人名义投标或弄虚作假骗取中标的行为处理如下：

1. **行政处罚**：
   - 投标人有下列情形之一的，属于弄虚作假骗取中标，将受到相应处罚（具体条款未明确处罚幅度，但属于严重违法行为）：
     (1) 使用伪造、变造的许可证件；
     (2) 3年内2次以上使用他人名义投标；
     (3) 弄虚作假骗取中标给招标人造成直接经济损失30万元以上；
     (4) 其他情节严重的行为。

2. **加重处罚**：
   - 若投标人在前次处罚执行期满后3年内再次实施同类违法行为，或情节特别严重的，**由工商行政管理机关吊销营业执照**。

3. **刑事责任**：
   - 若行为构成犯罪（如诈骗、伪造证件等），将**依法追究刑事责任**。

4. **关联责任**：
   - 出让或出租资格、资质证书供他人投标的，也将面临行政处罚或刑事责任（第六十九条）。

依据上述规定，处理措施包括罚款、吊销营业执照及刑事追责，具体适用取决于违法情节的严重程度和是否构成犯罪。